# Notebook #2: Limpieza: limpieza y transformación de los datos

In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../../src')
import funciones as f
import soporte_prep as sp

import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')
pd.options.display.float_format = '{:.2f}'.format
pd.set_option('display.max_columns', None)

In [2]:
df_infracciones_1 = pd.read_excel("C:\\Users\\DELL\\Downloads\\Infracciones de Tránsito, DIGESETT, 2010 - 2017.xlsx")
df_infracciones_2 = pd.read_excel("C:\\Users\\DELL\\Downloads\\Infracciones de Tránsito, DIGESETT, 2018 - 2024.xlsx")

In [ ]:
df_infracciones_1.columns = ['infracciones', 'total', 'anio']
df_infracciones_2.columns = df_infracciones_2.columns.str.lower() #pongo los nombres en minúsculas
df_infracciones_2.rename(columns={'año':'anio'}, inplace=True)

In [10]:
df_infracciones_2['total'] = df_infracciones_2.iloc[:,1:13].sum(axis=1) # creo la columna de total

In [11]:
df_infracciones_2.reset_index(drop=True, inplace=True)

In [ ]:
df_infracciones_2.columns = df_infracciones_2.columns.str.strip() # quito posibles espacios no vistos

In [27]:
df_infracciones_1['infracciones'] = df_infracciones_1['infracciones'].replace({
    'TRANSITAR EN VIA CONTRARIA': 'VIA CONTRARIA'
})

In [ ]:
# Calculo el porcentaje dentro de cada año usando transform
df_infracciones_1['porcentaje'] = df_infracciones_1.groupby('anio')['total'].transform(lambda x: (x / x.sum()) * 100)

In [29]:
df_infracciones_1['infracciones'] = df_infracciones_1['infracciones'].str.strip()
df_infracciones_1['infracciones'] = df_infracciones_1['infracciones'].str.lower()

In [30]:
df_infracciones_1= df_infracciones_1[['anio', 'infracciones', 'total', 'porcentaje']]

In [31]:
df_infracciones_1['infracciones'] = df_infracciones_1['infracciones'].replace({
    'estacionarse en lugares prohibido': 'estacionarse en lugares prohibidos'})

In [32]:
df_infracciones_1['infracciones'] = df_infracciones_1['infracciones'].replace({
    'licencia vencida': 'conducir sin licencia y/o vencida', 
    'conducir sin licencia': 'conducir sin licencia y/o vencida'})

In [ ]:
# calculo el porcentaje dentro de cada año usando transform
df_infracciones_2['porcentaje'] = df_infracciones_2.groupby('anio')['total'].transform(lambda x: (x / x.sum()) * 100)

# Filtro para quedarme con los valores totales a nivel anual
df_infracciones_2 = df_infracciones_2[['anio', 'infracciones', 'total', 'porcentaje']]

In [ ]:
# Quito posibles elementos que no me permitan trabajar la columna correctamente
df_infracciones_2['infracciones'] = df_infracciones_2['infracciones'].str.replace(
    r'^\d+\.?\s*|\(.*?\)', '', regex=True
).str.strip()

In [ ]:
df_infracciones_2['porcentaje'] = df_infracciones_2['porcentaje'].round(2)
df_infracciones_2['infracciones']=df_infracciones_2['infracciones'].str.replace('-', '')
df_infracciones_2['infracciones'] = df_infracciones_2['infracciones'].str.lower()

In [ ]:
mapeo_multiple = {
    # Estacionarse en lugares prohibidos
    'estacionarse en lugar prohibido': 'estacionarse en lugares prohibidos',
    'estacionarse en lugares prohibido': 'estacionarse en lugares prohibidos',
    'estacionarse en lugares prohibidos': 'estacionarse en lugares prohibidos',
    
    # Obstruir el tránsito
    'parar en la intersecciones / obstruir el tránsito': 'obstruir el transito',
    'obstruir el tránsito': 'obstruir el transito',
    
    # Conducir sin licencia y/o vencida
    'transitar sin licencia y/o vencida': 'conducir sin licencia y/o vencida',
    'transitar sin permiso': 'conducir sin licencia y/o vencida',
    'conducir sin licencia': 'conducir sin licencia y/o vencida',
    'licencia vencida': 'conducir sin licencia y/o vencida',
    'transitar sin licencia o vencida': 'conducir sin licencia y/o vencida',
    
    # Vía contraria
    'transitar en vía contraria': 'via contraria',
    'vía contraria': 'via contraria',
    
    # Hablar por celular
    'uso de celular o dispositivos electrónicos': 'hablar por celular',
    'uso del celular o dispositivos electrónicos  art. 221)': 'hablar por celular',
    
    # Transitar sin cinturón
    'transitar sin cinturón': 'transitar sin cinturon',
    'no uso del cinturón de seguridad': 'transitar sin cinturon',
    
    # Transitar sin casco
    'transitar sin casco protector': 'transitar sin casco',
    'no uso del casco protector': 'transitar sin casco',
    
    # Violar la luz roja
    'violar luz roja': 'violar la luz roja',
    'respeto a las señales del semáforo': 'violar la luz roja',
    
    # Giro prohibido
    'realizar giro prohibido': 'giro prohibido'
}

In [ ]:
# Quito espacios y mapeo la columna para reemplazar esos valores a su categoría correspondiente
df_infracciones_2['infracciones'] = df_infracciones_2['infracciones'].str.strip()
df_infracciones_2 = f.reemplazar_valores(
    dataframe=df_infracciones_2, 
    columna='infracciones', 
    mapeo=mapeo_multiple
)

In [ ]:
infracciones_validas = ['violar la luz roja', 'transitar sin casco',
       'transitar sin cinturon', 'hablar por celular', 'via contraria',
       'conducir sin licencia y/o vencida',
       'obstruir el transito', 'estacionarse en lugares prohibidos',
       'giro prohibido']

# Convertir todas las infracciones que no están en la lista de infracciones válidas a 'otros'
df_infracciones_2['infracciones'] = df_infracciones_2['infracciones'].apply(
    lambda x: x if x in infracciones_validas else 'otros'
)

In [41]:
df_infracciones_1.to_csv('../../Datos/infracciones/df_infracciones_1.csv')
df_infracciones_2.to_csv('../../Datos/infracciones/df_infracciones_2.csv')